In [1]:
# Import libraries
import torch
import torchvision.models
from torch import nn
from torchinfo import summary
from torchvision import transforms
from torchvision.models import ResNet152_Weights

from src.data_loader import create_dataloaders
from src.models.vgg_16 import VGG16
from src.utils import set_device_agnostic_mode
from src.training import train_model

/opt/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <CDAC6E34-8608-3E70-8B2F-32BCD38E90FB> /opt/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Initialize training parameters and get device

train_dir = 'data/train'
val_dir = 'data/val'

COLOR_CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 50

# Get a device to use for training/inference
device = set_device_agnostic_mode()

In [3]:
# Create transform for data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
    transforms.RandomAffine(degrees=40, translate=None, scale=(1, 2), shear=15),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Create training and validation data loaders
train_dataloader, val_dataloader, classes = create_dataloaders(train_dir, val_dir, transform, 
                                                               transform, BATCH_SIZE)

In [4]:
# Craete VGG16 CNN model
vgg_16_model = VGG16(COLOR_CHANNELS, len(classes))
print(summary(vgg_16_model, input_size=(BATCH_SIZE, COLOR_CHANNELS, 224, 224)))

vgg_16_model = vgg_16_model.to(device)

Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [32, 102]                 --
├─Sequential: 1-1                        [32, 64, 224, 224]        --
│    └─Conv2d: 2-1                       [32, 64, 224, 224]        1,792
│    └─BatchNorm2d: 2-2                  [32, 64, 224, 224]        128
│    └─ReLU: 2-3                         [32, 64, 224, 224]        --
├─Sequential: 1-2                        [32, 64, 112, 112]        --
│    └─Conv2d: 2-4                       [32, 64, 224, 224]        36,928
│    └─BatchNorm2d: 2-5                  [32, 64, 224, 224]        128
│    └─ReLU: 2-6                         [32, 64, 224, 224]        --
│    └─MaxPool2d: 2-7                    [32, 64, 112, 112]        --
├─Sequential: 1-3                        [32, 128, 112, 112]       --
│    └─Conv2d: 2-8                       [32, 128, 112, 112]       73,856
│    └─BatchNorm2d: 2-9                  [32, 128, 112, 112]       256
│

In [7]:
# Create optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg_16_model.parameters(), lr=0.075)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Train model
train_model(
    EPOCHS,
    len(classes),
    vgg_16_model,
    train_dataloader,
    val_dataloader,
    loss_fn,
    optimizer,
    scheduler,
    device
)

Epoch 1, Train Loss 4.276609420776367, Train Accuracy 0.05359455943107605, Val Loss 4.421390533447266, Val Accuracy 0.0234375
Epoch 2, Train Loss 3.969855546951294, Train Accuracy 0.07901554554700851, Val Loss 5.907750129699707, Val Accuracy 0.0263671875


KeyboardInterrupt: 

In [4]:
# Get resnet152 params and transforms
resnet152_weights = ResNet152_Weights.DEFAULT
auto_transforms = resnet152_weights.transforms()

In [5]:
# Create training and validation data loaders
train_dataloader, val_dataloader, classes = create_dataloaders(train_dir, val_dir, 
                                                               auto_transforms, auto_transforms, BATCH_SIZE)

6149
1020


In [6]:
# Initialize Resnet 152 model and fine tune classification layer
resnet152_model = torchvision.models.resnet152(weights=resnet152_weights)

for name, layer in resnet152_model.named_children():
    if name not in ['fc']:
        for param in layer.parameters():
            param.requires_grad = False

resnet152_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)
print(summary(resnet152_model,
        input_size=(BATCH_SIZE, COLOR_CHANNELS, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"]))
resnet152_model = resnet152_model.to(device)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
ResNet                                   [32, 3, 224, 224]         [32, 102]                 --                        Partial
├─Conv2d: 1-1                            [32, 3, 224, 224]         [32, 64, 112, 112]        (9,408)                   False
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        [32, 64, 112, 112]        (128)                     False
├─ReLU: 1-3                              [32, 64, 112, 112]        [32, 64, 112, 112]        --                        --
├─MaxPool2d: 1-4                         [32, 64, 112, 112]        [32, 64, 56, 56]          --                        --
├─Sequential: 1-5                        [32, 64, 56, 56]          [32, 256, 56, 56]         --                        False
│    └─Bottleneck: 2-1                   [32, 64, 56, 56]          [32, 256, 56, 56]         --                        False


In [ ]:
# Create optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(resnet152_model.parameters(), lr=0.001)

optimizer = torch.optim.Adam(
    [
        {'params': resnet152_model.conv1.parameters(),  'lr': 0.000001},
        {'params': resnet152_model.layer1.parameters(), 'lr': 0.000001},
        {'params': resnet152_model.layer2.parameters(), 'lr': 0.00001},
        {'params': resnet152_model.layer3.parameters(), 'lr': 0.00001},
        {'params': resnet152_model.layer4.parameters(), 'lr': 0.0001},
        {'params': resnet152_model.fc.parameters(),     'lr': 0.001}
    ], lr=0.0, weight_decay=0.001)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Train model
train_model(
    EPOCHS,
    len(classes),
    resnet152_model,
    train_dataloader,
    val_dataloader,
    loss_fn,
    optimizer,
    scheduler,
    device
)

Epoch 1, Train Loss 2.5453481674194336, Train Accuracy 0.5545984506607056, Val Loss 1.9465062618255615, Val Accuracy 0.7021484375
Epoch 2, Train Loss 0.9353354573249817, Train Accuracy 0.9068976640701294, Val Loss 1.3322378396987915, Val Accuracy 0.8271484375
Epoch 3, Train Loss 0.5645363330841064, Train Accuracy 0.9499028921127319, Val Loss 0.9797909259796143, Val Accuracy 0.8533760905265808
Epoch 4, Train Loss 0.4199342429637909, Train Accuracy 0.9679403901100159, Val Loss 0.8598898649215698, Val Accuracy 0.8783482313156128
Epoch 5, Train Loss 0.3315292000770569, Train Accuracy 0.9814767241477966, Val Loss 0.8276596665382385, Val Accuracy 0.8713727593421936
Epoch 6, Train Loss 0.28458479046821594, Train Accuracy 0.987532377243042, Val Loss 0.7514386773109436, Val Accuracy 0.8754185438156128
